## Data Cleaning & Feature Engineering

---

### Objective

In this notebook, we clean the raw loan dataset and prepare it for machine learning.
We will **inspect missing values**, **justify each decision**, **apply transformations**, and **verify results after every step**.

This step-by-step approach ensures transparency and prevents silent data leakage or errors.

## 1️⃣ Load Libraries & Data


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)

In [4]:
train_df = pd.read_csv('../data/raw/loan-train.xls')
test_df  = pd.read_csv('../data/raw/loan-test.xls')

print('Train shape:', train_df.shape)
print('Test shape:', test_df.shape)

Train shape: (614, 13)
Test shape: (367, 12)


## 2️⃣ Initial Data Inspection

### Preview the data


In [5]:
train_df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


## 3️⃣ Missing Value Analysis

### Count missing values per column

In [7]:
missing_count = train_df.isnull().sum()
missing_pct = (missing_count / len(train_df)) * 100

missing_df = pd.DataFrame({
    'Missing Count': missing_count,
    'Missing %': missing_pct
}).sort_values('Missing %', ascending=False)

missing_df

,Missing Count,Missing %
Credit_History,50,8.143322
Self_Employed,32,5.211726
LoanAmount,22,3.583062
Dependents,15,2.442997
Loan_Amount_Term,14,2.280130
Gender,13,2.117264
Married,3,0.488599
Loan_ID,0,0.000000
Education,0,0.000000
ApplicantIncome,0,0.000000


### Interpretation (Markdown)

* Missing values are **below 5%** for all affected features
* Dataset is **small (~600 rows)**
* Dropping rows would result in unnecessary information loss

✅ **Decision: Impute missing values instead of dropping rows**

## 4️⃣ Drop Identifier Column

### Why drop Loan_ID?

* It is a **primary key**, not predictive
* Unique per row → cannot generalize

In [8]:
train_df.drop(columns=['Loan_ID'], inplace=True)
test_df.drop(columns=['Loan_ID'], inplace=True)

In [9]:
train_df.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


## 5️⃣ Separate Feature Types

In [10]:
categorical_cols = train_df.select_dtypes(include='object').columns.drop('Loan_Status')
numerical_cols = train_df.select_dtypes(exclude='object').columns

print('Categorical Columns:', list(categorical_cols))
print('Numerical Columns:', list(numerical_cols))

Categorical Columns: ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Property_Area']
Numerical Columns: ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History']


## 6️⃣ Impute Categorical Features (Mode)

### Why mode?

* Categorical variables have no mean
* Mode preserves most common category

In [11]:
for col in categorical_cols:
    mode_value = train_df[col].mode()[0]
    train_df[col].fillna(mode_value, inplace=True)
    test_df[col].fillna(mode_value, inplace=True)

### Verify

In [12]:
train_df[categorical_cols].isnull().sum()

Gender           0
Married          0
Dependents       0
Education        0
Self_Employed    0
Property_Area    0
dtype: int64

## 7️⃣ Impute Numerical Features (Median)

### Why median?

* Income & loan amounts are skewed
* Median is robust to outliers


In [13]:
for col in numerical_cols:
    median_value = train_df[col].median()
    train_df[col].fillna(median_value, inplace=True)
    test_df[col].fillna(median_value, inplace=True)

### Verify

In [14]:
train_df[numerical_cols].isnull().sum()

ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
dtype: int64

## 8️⃣ Final Missing Value Check
---

In [15]:
train_df.isnull().sum()

Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

## 9️⃣ Feature Engineering

### 9.1 Total Income

In [16]:
train_df['TotalIncome'] = train_df['ApplicantIncome'] + train_df['CoapplicantIncome']
test_df['TotalIncome'] = test_df['ApplicantIncome'] + test_df['CoapplicantIncome']

In [17]:
train_df[['ApplicantIncome','CoapplicantIncome','TotalIncome']].head()

,ApplicantIncome,CoapplicantIncome,TotalIncome
0,5849,0.0,5849.0
1,4583,1508.0,6091.0
2,3000,0.0,3000.0
3,2583,2358.0,4941.0
4,6000,0.0,6000.0


### 9.2 EMI (Loan Burden)

In [18]:
train_df['EMI'] = train_df['LoanAmount'] / train_df['Loan_Amount_Term']
test_df['EMI'] = test_df['LoanAmount'] / test_df['Loan_Amount_Term']

In [19]:
train_df[['LoanAmount','Loan_Amount_Term','EMI']].head()

,LoanAmount,Loan_Amount_Term,EMI
0,128.0,360.0,0.355556
1,128.0,360.0,0.355556
2,66.0,360.0,0.183333
3,120.0,360.0,0.333333
4,141.0,360.0,0.391667


## 🔟 Skewness Check & Log Transformation

In [20]:
train_df[['ApplicantIncome','CoapplicantIncome','LoanAmount','TotalIncome']].skew()

ApplicantIncome      6.539513
CoapplicantIncome    7.491531
LoanAmount           2.743053
TotalIncome          5.633449
dtype: float64

### Apply log1p transformation

In [21]:
for col in ['ApplicantIncome','CoapplicantIncome','LoanAmount','TotalIncome']:
    train_df[col] = np.log1p(train_df[col])
    test_df[col] = np.log1p(test_df[col])

## 1️⃣1️⃣ Encode Target Variable

In [22]:
train_df['Loan_Status'] = train_df['Loan_Status'].map({'Y':1, 'N':0})

In [23]:
train_df['Loan_Status'].value_counts()

Loan_Status
1    422
0    192
Name: count, dtype: int64

In [28]:
train_df

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status,TotalIncome,EMI,Gender_Male,Married_Yes,Dependents_1,Dependents_2,Dependents_3+,Education_Not Graduate,Self_Employed_Yes,Property_Area_Semiurban,Property_Area_Urban
0,8.674197,0.000000,4.859812,360.0,1.0,1,8.674197,0.355556,True,False,False,False,False,False,False,False,True
1,8.430327,7.319202,4.859812,360.0,1.0,0,8.714732,0.355556,True,True,True,False,False,False,False,False,False
2,8.006701,0.000000,4.204693,360.0,1.0,1,8.006701,0.183333,True,True,False,False,False,False,True,False,True
3,7.857094,7.765993,4.795791,360.0,1.0,1,8.505525,0.333333,True,True,False,False,False,True,False,False,True
4,8.699681,0.000000,4.955827,360.0,1.0,1,8.699681,0.391667,True,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,7.972811,0.000000,4.276666,360.0,1.0,1,7.972811,0.197222,False,False,False,False,False,False,False,False,False
610,8.320448,0.000000,3.713572,180.0,1.0,1,8.320448,0.222222,True,True,False,False,True,False,False,False,False
611,8.996280,5.484797,5.537334,360.0,1.0,1,9.025576,0.702778,True,True,True,False,False,False,False,False,True
612,8.933796,0.000000,5.236442,360.0,1.0,1,8.933796,0.519444,True,True,False,True,False,False,False,False,True


## 1️⃣2️⃣ One-Hot Encode Categorical Features

In [24]:
train_df = pd.get_dummies(train_df, drop_first=True)
test_df = pd.get_dummies(test_df, drop_first=True)

### Align train & test

In [25]:
train_df, test_df = train_df.align(test_df, join='left', axis=1, fill_value=0)

In [29]:
train_df

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status,TotalIncome,EMI,Gender_Male,Married_Yes,Dependents_1,Dependents_2,Dependents_3+,Education_Not Graduate,Self_Employed_Yes,Property_Area_Semiurban,Property_Area_Urban
0,8.674197,0.000000,4.859812,360.0,1.0,1,8.674197,0.355556,True,False,False,False,False,False,False,False,True
1,8.430327,7.319202,4.859812,360.0,1.0,0,8.714732,0.355556,True,True,True,False,False,False,False,False,False
2,8.006701,0.000000,4.204693,360.0,1.0,1,8.006701,0.183333,True,True,False,False,False,False,True,False,True
3,7.857094,7.765993,4.795791,360.0,1.0,1,8.505525,0.333333,True,True,False,False,False,True,False,False,True
4,8.699681,0.000000,4.955827,360.0,1.0,1,8.699681,0.391667,True,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,7.972811,0.000000,4.276666,360.0,1.0,1,7.972811,0.197222,False,False,False,False,False,False,False,False,False
610,8.320448,0.000000,3.713572,180.0,1.0,1,8.320448,0.222222,True,True,False,False,True,False,False,False,False
611,8.996280,5.484797,5.537334,360.0,1.0,1,9.025576,0.702778,True,True,True,False,False,False,False,False,True
612,8.933796,0.000000,5.236442,360.0,1.0,1,8.933796,0.519444,True,True,False,True,False,False,False,False,True


## 1️⃣3️⃣ Feature Scaling

In [ ]:
# Import necessary libraries
from sklearn.preprocessing import StandardScaler

# Separate features and target from training data
X = train_df.drop('Loan_Status', axis=1)
y = train_df['Loan_Status']

# Make sure test has NO target column
X_test = test_df.copy()

# First, check what columns are in your test dataframe
print("Test columns:", X_test.columns.tolist())
print("Training features columns:", X.columns.tolist())

# If 'Loan_Status' is in test dataframe, drop it
if 'Loan_Status' in X_test.columns:
    X_test = X_test.drop('Loan_Status', axis=1)
    print("Dropped 'Loan_Status' from test data")

# Check column alignment
print(f"\nAre columns in the same order? {list(X.columns) == list(X_test.columns)}")
print(f"Are column sets the same? {set(X.columns) == set(X_test.columns)}")

# Reorder test columns to match training columns (important for sklearn)
X_test = X_test[X.columns]

# Now proceed with scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)  # This should work now

print(f"\nScaling completed successfully!")
print(f"X_scaled shape: {X_scaled.shape}")
print(f"X_test_scaled shape: {X_test_scaled.shape}")

Test columns: ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History', 'Loan_Status', 'TotalIncome', 'EMI', 'Gender_Male', 'Married_Yes', 'Dependents_1', 'Dependents_2', 'Dependents_3+', 'Education_Not Graduate', 'Self_Employed_Yes', 'Property_Area_Semiurban', 'Property_Area_Urban']
Training features columns: ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History', 'TotalIncome', 'EMI', 'Gender_Male', 'Married_Yes', 'Dependents_1', 'Dependents_2', 'Dependents_3+', 'Education_Not Graduate', 'Self_Employed_Yes', 'Property_Area_Semiurban', 'Property_Area_Urban']
Dropped 'Loan_Status' from test data

Are columns in the same order? True
Are column sets the same? True

Scaling completed successfully!
X_scaled shape: (614, 16)
X_test_scaled shape: (367, 16)


## 1️⃣4️⃣ Final Shape Check

In [35]:
print('X_train:', X_scaled.shape)
print('X_test:', X_test_scaled.shape)
print('y_train:', y.shape)

X_train: (614, 16)
X_test: (367, 16)
y_train: (614,)


## 1️⃣5️⃣ Save Processed Data

In [37]:
pd.DataFrame(X_scaled, columns=X.columns).to_csv('../data/processed/X_train.csv', index=False)
pd.DataFrame(X_test_scaled, columns=X.columns).to_csv('../data/processed/X_test.csv', index=False)
y.to_csv('../data/processed/y_train.csv', index=False)